# Xử lý dữ liệu comment

In [1]:
import requests
import pandas as pd
import numpy as np

In [ ]:
API = ""
with open("API.txt", 'r') as file : 
    API = file.read()

## Mô hình summary 
- Dùng để tóm tắt bình luận có dài vượt quá đầu vào của mô hình sentiment

In [44]:
API_URL = "https://api-inference.huggingface.co/models/pengold/t5-vietnamese-summarization"
headers = {"Authorization": API}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

def summary(sentence) :
	if (len(sentence) <= 256) : return sentence 	
	output = query({
		"inputs": sentence,
	})
	
	return output[0]['summary_text']

## Mô hình đánh gía cảm xúc bình luận người dùng

In [45]:
# Model dự đoán sentiment bình luận người dùng
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)


def sentiment_analyst(sentence) : 
	
	input_ids = torch.tensor([tokenizer.encode(sentence)])

	with torch.no_grad():
		out = model(input_ids)
		res_lst = out.logits.softmax(dim=-1).tolist()
	res = "Negative"
	m = res_lst[0][0]
	if m < res_lst[0][1] : 
		res = 'Positive'
		m = res_lst[0][1]
	
	if m < res_lst[0][2] : 
		res = 'Neutral'
		m = res_lst[0][2]
	return res


In [46]:
comment_df = pd.read_csv('Data\comment_summary.csv')
comment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6885 entries, 0 to 6884
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   post_id                 6885 non-null   int64 
 1   comment_id              6885 non-null   int64 
 2   comment_url             6885 non-null   object
 3   commenter_id            6885 non-null   int64 
 4   commenter_url           6885 non-null   object
 5   comment_text            6885 non-null   object
 6   comment_time            6527 non-null   object
 7   like                    6885 non-null   int64 
 8   haha                    6885 non-null   int64 
 9   sad                     6885 non-null   int64 
 10  love                    6885 non-null   int64 
 11  care                    6885 non-null   int64 
 12  wow                     6885 non-null   int64 
 13  angry                   6885 non-null   int64 
 14  comment_reaction_count  6885 non-null   int64 
 15  is_r

In [47]:
comment_df.iloc[65]

post_id                                                     737500015083384
comment_id                                                 1021589905751270
comment_url                           https://facebook.com/1021589905751270
commenter_id                                                100064108125602
commenter_url             https://facebook.com/profile.php?id=1000641081...
comment_text              Hiếu Dương nói như ông, ông hỏi thử xem tại sa...
comment_time                                            2023-11-09 00:00:00
like                                                                      0
haha                                                                      0
sad                                                                       0
love                                                                      0
care                                                                      0
wow                                                                       0
angry       

In [ ]:
tmp_1 = comment_df.loc[:3000, "comment_text"].apply(lambda x : summary(x))


In [ ]:
tmp_1

0       Đăng mic check vs TL đi chứ đánh với anh em mộ...
1                            Nghe nói fiddlesticks cút hả
2       Hôm nghe bảo về fam lại mà mấy bố kêu về fam n...
3               Anh Khánh đi bộ nai xừ chưa về live nữa 😂
4                                             Dương Phong
                              ...                        
2996                           Thiệu Vĩ Khang thì sao ???
2997                      Hoang Anh đúng chân đèn có khác
2998       Minh Hoàng :)))) bảo đội hay hơn thắng đi :)))
2999    Hoang Anh ủa t bảo t fan tw hả =))) m bảo vả n...
3000        Minh Hoàng t nói mày fan TW lúc nào đấy, :)),
Name: comment_text, Length: 3001, dtype: object

In [ ]:
tmp_2 = comment_df.loc[3000:, "comment_text"].apply(lambda x : summary(x))

In [ ]:
tmp_2[1:]

3001    Hoang Anh đúng chân đèn sì, clb nào fan đấy =))))
3002                           Hùng Thanh Nguyễn đúng thế
3003    thể thao thắng thua bth,có đã từng thắng lpl c...
3004    TriỀu Nguyễn nói thẳng luôn trình độ việt nam ...
3005    TriỀu Nguyễn gì vậy cha nội, vậy sao không bảo...
                              ...                        
6880                                   Khi nào đámh ad oi
6881    nghe hừng hực khí thế luôn. cố lên nào mn. VCS...
6882                                 Mai mấyh đánh thế mn
6883    Các thành viên đừng để bất kỳ vấn đề nào ảnh h...
6884                     Điện Lạnh An Phát-tài mai ông oi
Name: comment_text, Length: 3884, dtype: object

In [ ]:
comment_df['comment_text'] = pd.concat([tmp_1, tmp_2[1:]])
comment_df

,comment_id,comment_url,commenter_id,commenter_url,comment_text,comment_time,like,haha,sad,love,care,wow,angry,comment_reaction_count,is_reply
0,687445613484826,https://facebook.com/687445613484826,100032860689002,https://facebook.com/profile.php?id=1000328606...,Đăng mic check vs TL đi chứ đánh với anh em mộ...,2023-11-14 00:00:00,6,0,0,0,0,0,0,6,False
1,6766375476749289,https://facebook.com/6766375476749289,100043315503165,https://facebook.com/profile.php?id=1000433155...,Nghe nói fiddlesticks cút hả,2023-11-09 00:00:00,0,0,0,0,0,0,0,0,False
2,889568115848198,https://facebook.com/889568115848198,100008777650405,https://facebook.com/khuongkts?eav=AfYOQ02kFte...,Hôm nghe bảo về fam lại mà mấy bố kêu về fam n...,2023-11-15 00:00:00,0,0,0,0,0,0,0,0,False
3,874599703945064,https://facebook.com/874599703945064,61552993208509,https://facebook.com/profile.php?id=6155299320...,Anh Khánh đi bộ nai xừ chưa về live nữa 😂,2023-11-13 00:00:00,0,0,0,0,0,0,0,0,False
4,853331139670600,https://facebook.com/853331139670600,100069500763685,https://facebook.com/profile.php?id=1000695007...,Dương Phong,2023-11-15 00:00:00,0,0,0,0,0,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6880,282538447954985,https://facebook.com/282538447954985,100004381742027,https://facebook.com/tai.truong.3538039?eav=Af...,Khi nào đámh ad oi,2023-10-08,0,0,0,0,0,0,0,0,False
6881,1070332167460223,https://facebook.com/1070332167460223,100088597435713,https://facebook.com/profile.php?id=1000885974...,nghe hừng hực khí thế luôn. cố lên nào mn. VCS...,2023-10-08,0,0,0,0,0,0,0,0,False
6882,857493452829263,https://facebook.com/857493452829263,100047301279362,https://facebook.com/profile.php?id=1000473012...,Mai mấyh đánh thế mn,2023-10-08,0,0,0,0,0,0,0,0,False
6883,690150839830092,https://facebook.com/690150839830092,100037243068003,https://facebook.com/profile.php?id=1000372430...,Các thành viên đừng để bất kỳ vấn đề nào ảnh h...,2023-10-08,0,0,0,0,0,0,0,0,False


In [ ]:
comment_df[comment_df['comment_text'].str.len() > 256]['comment_text']

Series([], Name: comment_text, dtype: object)

In [ ]:
comment_df['sentiment'] = comment_df['comment_text'].apply(lambda x : sentiment_analyst(x))

In [ ]:
comment_df['sentiment'].isna().sum()

0

In [ ]:
comment_df.to_csv('Data\comment_summary_solv.csv', index = False)